In [1]:
from tinyshift.series import trend_significance
from utilsforecast.preprocessing import fill_gaps
from utils import remove_leading_zeros, is_obsolete
import numpy as np
from statsforecast.models import AutoARIMA, Naive, AutoETS, AutoTheta, AutoCES, AutoMFLES
from utilsforecast.losses import rmse, mae, bias, cfe, smape
from statsforecast import StatsForecast
from utilsforecast.evaluation import evaluate
import pandas as pd

/home/heylucasleao/forecasting/.venv/lib/python3.10/site-packages/fs/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)  # type: ignore
/home/heylucasleao/forecasting/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv'
df = pd.read_csv(url, parse_dates=['Month'])
df["unique_id"] = "1"
df.rename(columns={"Month": "ds", "Passengers": "y"}, inplace=True)
df = fill_gaps(df, freq="ME", end="per_serie", id_col="unique_id", time_col="ds")
df = df.groupby("unique_id")[df.columns].apply(remove_leading_zeros).reset_index(drop=True)
days_obsoletes=180
obsolete_series = df.groupby("unique_id")[df.columns].apply(is_obsolete, days_obsoletes)
obsolote_ids = obsolete_series[obsolete_series].index.tolist()
assert len(obsolote_ids) == 0, f"Obsolete series found: {obsolote_ids}"
df.groupby("unique_id")["y"].apply(trend_significance)

unique_id
1    (0.853638165603188, 4.020274506593391e-61)
Name: y, dtype: object

In [3]:
seasonal_length = 12
sf = StatsForecast(
    models=[
        AutoARIMA(season_length=seasonal_length),
        AutoETS(season_length=seasonal_length),
        AutoTheta(season_length=seasonal_length),
        AutoCES(season_length=seasonal_length),
        AutoMFLES(season_length=seasonal_length, test_size=3, n_windows=5, metric = "mae"),
    ],
    freq='MS',
    fallback_model=Naive(),
    n_jobs=-1
)

In [4]:
horizon = 12
train = df[:-horizon]
test = df[-horizon:]

In [5]:
fc = sf.forecast(df=train[["unique_id", "ds", "y"]], h=horizon) #fit_predict

In [6]:
def wape(df, models, id_col='unique_id', target_col='y'):
    """
    Calcula o WAPE (Weighted Absolute Percentage Error).
    'df' é o DataFrame de avaliação com 'unique_id', 'ds', 'y' (target) e colunas de modelo.
    """
    actual = df[target_col].to_numpy()

    results = {}
    for model_name in models:
        forecast = df[model_name].to_numpy()

        numerator = np.sum(np.abs(forecast - actual))
        denominator = np.sum(np.abs(actual))

        if denominator == 0:
            wape_score = 0.0 
        else:
            wape_score = numerator / denominator

        results[model_name] = wape_score

    return pd.DataFrame({
        'unique_id': df[id_col].unique(),
        'metric': 'wape',
        **results
    })

In [7]:
metrics = [
    mae,
    rmse,
    bias,
    cfe,
    wape,
    smape
]

models = ["AutoARIMA", "AutoETS", "AutoTheta", "CES", "SCUM", "AutoMFLES"]
scum = ["AutoARIMA", "AutoETS", "AutoTheta", "CES"]
fc["SCUM"] = fc[scum].median(axis=1)
fc.loc[fc["SCUM"] < 1e-1, "SCUM"] = 0

In [8]:
test = pd.merge(test, fc, on=["unique_id", "ds"], how="inner")
columns = ["unique_id", 
               "ds", 
               "y", 
               "AutoARIMA", 
               "AutoETS", 
               "AutoTheta", 
               "CES", 
               "SCUM", "AutoMFLES"]

In [9]:
evaluate(test[columns], 
         metrics=metrics, 
         models=models, 
         id_col="unique_id", 
         time_col="ds", 
         target_col="y")

,unique_id,metric,AutoARIMA,AutoETS,AutoTheta,CES,SCUM,AutoMFLES
0,1,mae,18.515821,35.612475,19.357396,10.145073,12.008021,15.627861
1,1,rmse,23.919481,40.083621,24.985159,14.657908,16.899254,21.679438
2,1,bias,16.968961,-27.608716,-9.096141,5.823789,-2.396108,14.260185
3,1,cfe,203.627533,-331.304596,-109.153687,69.885468,-28.753296,171.122223
4,1,wape,0.038885,0.074790,0.040653,0.021306,0.025218,0.032820
5,1,smape,0.020156,0.037253,0.019655,0.010630,0.012573,0.016938
